# Crash Report Jan 18, 2016


## Step 1: Setup

In [3]:
"""
General import statements
"""
import crash_analysis
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline

In [3]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReports\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [4]:
"""
Save/load dataframe to/from CSV
"""
new_data = False
cache_location = 'src/data/2016040014.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = crash_analysis.read_csv(cache_location)

In [5]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)

print(col_names)
print(versions)
print(num_crashes)

['ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml']
2016040014    5085
Name: AppVersion, dtype: int64
5085


In [6]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)
print

total words: 7650
instal     	:  463 	 [['installing'], ['installing'], ['installing'], ['installed'], ['install'], ['installing']]
open       	:  349 	 [['opening'], ['opening'], ['opening'], ['opening'], ['opened'], ['opening']]
2016       	:  318 	 [['2016'], ['2016'], ['2016'], ['2016'], ['2016'], ['2016']]
program    	:  254 	 [['program'], ['program'], ['program'], ['program'], ['program'], ['program']]
client     	:  244 	 [['clients'], ['clients'], ['client'], ['client'], ['client'], ['client']]
updat      	:  230 	 [['updating'], ['updates'], ['updates'], ['updates'], ['updates'], ['updates']]
proseri    	:  217 	 [['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries']]
file       	:  217 	 [['file'], ['files'], ['file'], ['file'], ['file'], ['file']]
tri        	:  191 	 [['trying'], ['trying'], ['trying'], ['trying'], ['trying'], ['trying']]
download   	:  166 	 [['downloading'], ['downloading'], ['downloading'], ['downloading'], ['downloadi

In [20]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe


def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType', 'WorkStationType', 'OperatingSystem']):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()


    total = sum(field2_hist)

    print("Query {0} \t count: {1} \t {2} ({3}): {4}".format(n+1, count, field1, total, top_key.encode('ascii', 'ignore')))
    
    print(field2_hist)
    print '\n'

# [print_parings(n, field1='ExceptionModuleBase', groupings=['ExceptionAddress', 'ProblemDescription']) for n in range(1, 5)]
# [print_parings(n, groupings='CustNum') for n in [0]]
# print_parings(1, groupings=['CustNum', 'CrashGUID'])
# df['ExceptionAddress'].value_counts()
df[df['CustNum'] == '0000372659']['CrashGUID']

155     08be345c-23e4-40e5-85e3-63a9ee613895
715     2631e26b-c7b7-4c46-8d46-cd5ad781b5b8
1292    427c6354-5af9-4e24-b743-61f097c56312
1811    5e6c1664-198c-45cb-b48e-b4b1931f1ccb
1988    676bb613-13b3-45b0-8252-3e289efb9d81
2268    75f4b478-1942-461b-a853-5b76b41d6f45
2287    770749ef-9d0a-43cc-a77b-258ac55dc1cb
2421    7e95e61b-7432-423c-833d-913e9f4760dc
4553    e4c0ade5-0ec3-459b-b28d-8ff7c180e9a9
4714    ed03bfc9-eca5-43bf-bc89-baf0384be207
4727    edc3fdd2-9eb7-4b72-ba82-26a4bf1d7a9e
4820    f24b78cb-0be1-4a24-8235-fcde98289a6e
Name: CrashGUID, dtype: object

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception